1. Numerically simulate a Rayleigh multipath channel. Choose a transmitter location $(x_t, y_t)$, an array of locations $[x_r(t_m), y_r(t_m)]$ modeling a moving receiver, and random coordinates $(x_n, y_n)$ for Ns scatterers. The receiver should at least be a few wavelengths in total and a fraction of a wavelength in each time step. Model the electric field intensity at the receiver using $ E = \sum_{n=1}^{N_s} R_n e^{−jkr_n} /r_n$, where $R_n$ is a reflection coefficient and $r_n$ is the total path distance from the transmitter to the nth scatterer and then to the receiver. For a simple channel model, $R_n = 1$, or the reflection coefficient can be uniformly distributed with magnitude between zero and one.

(a) Plot the time history of the received power on a dB scale

In [ ]:
# imports
import numpy as np

# transmitter location
tx_loc = [0, 0]

# array of locations for receiver
rx_locs = np.array([[0,0],
                    [1,1],
                    [2,2],
                    [3,3],
                    [4,4]])